In [ ]:
!pip install pythainlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.9/17.9 MB 47.6 MB/s eta 0:00:00


In [12]:
import pandas as pd
import warnings
from sklearn.metrics import classification_report
from pythainlp.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [57]:
pos_text = ("C:/Users/Gumpun/Sentiment-Analysis-System-for-Consumer-Products/DataReviewProductThai/Positive.csv")
neg_text = ("C:/Users/Gumpun/Sentiment-Analysis-System-for-Consumer-Products/DataReviewProductThai/Negative.csv")
neu_text = ("C:/Users/Gumpun/Sentiment-Analysis-System-for-Consumer-Products/DataReviewProductThai/Neutrally.csv")

In [59]:
pos_df = pd.read_csv(pos_text, delimiter=',', encoding='utf-8').dropna()
neg_df = pd.read_csv(neg_text, delimiter=',', encoding='utf-8').dropna()
neu_df = pd.read_csv(neu_text, delimiter=',', encoding='utf-8').dropna()

data = pd.concat([pos_df, neg_df, neu_df])
data.sample(100)

,User,Message,Sentiment
1230,M***k,ใส่สบาย ผ้าบางตามราคา,Neutrally
451,****9,จัดส่งรวดเร็วมาก บรรจุหีบห่อดี คุณภาพสินค้าตาม...,Positive
995,N***�,สมราคา,Positive
399,Yew C.,โฆษณาเกินไปครับ ไม่ได้ออก ยอมซักมือดีฟ่า,Negative
637,Atchadaporn P.,สินค้าคุ้มค่ามากกกกกกกกกกกกก จัดส่งรวดเร็ว คุณ...,Positive
...,...,...,...
7,กีระติ แ.,สั่งรอบที่สองชุด ทำไมได้อุปกรณ์เสริมมาชุดเดียว...,Neutrally
1241,N***.,สินค้าไม่มีกล่อง แต่เป็นการแพ็คของด้วยบับโบ้ ส...,Neutrally
948,*******048,เล็กไปนิดหน่อย,Neutrally
762,วีกิจ ธ.,เป็นสินค้าคุณภาพดีเหมาะสมกับราคาสีสันย้อมดีใส่...,Positive


In [61]:
# สร้างฟังก์ชันสำหรับการลบอักขระที่ซ้ำกันในแต่ละคำเท่านั้น
def remove_duplicate_chars(text):
    if isinstance(text, str):  # ตรวจสอบว่าข้อความไม่ใช่ NaN
        unique_words = []
        words = word_tokenize(text)
        for word in words:
            unique_word = ''
            for char in word:
                if char not in unique_word:
                    unique_word += char
            unique_words.append(unique_word)
        return ' '.join(unique_words)
    else:
        return text  # ส่งค่า NaN กลับหากเป็น NaN

# ใช้ฟังก์ชัน `remove_duplicate_chars` เพื่อ Tokenize และลบคำที่ซ้ำ
data['Message'] = data['Message'].apply(remove_duplicate_chars)
data.sample(10)

,User,Message,Sentiment
284,R***.,เล็ก,Neutrally
726,Donlaya R.,ได้รับ ของ แล้ว นะค สิน ค้าส่ง ไว มาก ๆ ค่ะ ...,Positive
1359,งงอ๋อ'อ ฯ.,กล่อง มี รอย ตำหนิ นิดห่อย ครับ แต่ ของ ตรง ...,Neutrally
931,Wannee Y.,มาถึง กระจ เ เตก ใช้ไม่ด ค่ะ,Negative
1237,ฐานิดา ก.,คอกลม ทำให้ สบาย ลวดาย การ์ตูน น่ารัก มาก,Positive
1149,*******469,ไม่ ตรง ปก,Neutrally
444,ดามร ล.,ราค ถูก แต่ ผ้า บาง มาก,Neutrally
1409,ธนพร ป.,ได้รับ ของ ตามที่ ได้ แจ้ง ไว้ กระท สวย แพ...,Positive
570,*******605,สี ไม่ ตรง ปก ครับ สั่ง สี ดำ,Positive
953,เอก,คุณภาพ ตาม ราค นั่แหละ ครับ อย่า คาดหวัง เยอะ ...,Negative


In [89]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder

Message = data['Message'].values
Sentiment = data['Sentiment'].values

# ใช้สำหรับแปลงข้อความเป็นเวกเตอร์ของจำนวนคำที่ปรากฏในข้อความนั้น ๆ
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(Message).toarray()

# Encoding labels
encoder = LabelEncoder()
encoded_labels = encoder.fit_transform(Sentiment)

print("Encoded labels:", encoded_labels)
print("Mapping of encoded labels to original labels:")
for label, original_label in enumerate(encoder.classes_):
    print(f"{label}: {original_label}")

Encoded labels: [2 2 2 ... 1 1 1]
Mapping of encoded labels to original labels:
0: Negative
1: Neutrally
2: Positive


In [91]:
# แบ่งข้อมูลเป็นชุดฝึกและชุดทดสอบ
X_train, X_test, y_train, y_test = train_test_split(X, encoded_labels, stratify=encoded_labels, test_size=0.1, random_state=42)

In [93]:
#ใช้สำหรับการค้นหาค่าพารามิเตอร์ที่เหมาะสมที่สุดสำหรับโมเดล SVM
def grid_search_hyperparameters(X_train, y_train, X_test, y_test, hyperparameters):
    best_accuracy = 0
    best_hyperparameters = None

    for params in hyperparameters:
        learning_rate, lambda_param, num_epochs, batch_size = params

        # Initialize and train the SVM model with the current hyperparameters
        svm_model = SVM(learning_rate=learning_rate, lambda_param=lambda_param, num_epochs=num_epochs, batch_size=batch_size)
        svm_model.fit(X_train, y_train)

        # Make predictions on the test set
        predictions = svm_model.predict(X_test)

        # Calculate accuracy
        accuracy = accuracy_score(y_test, predictions)

        # Update the best hyperparameters if the current ones perform better
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_hyperparameters = params

    return best_hyperparameters, best_accuracy

In [95]:
class SVM:
    def __init__(self, learning_rate=0.0001, lambda_param=0.01, num_epochs=500, batch_size=32):
        self.learning_rate = learning_rate
        self.lambda_param = lambda_param
        self.num_epochs = num_epochs
        self.weights = None
        self.bias = None
        self.num_classes = None
        self.batch_size = batch_size

    def fit(self, X, y):
        num_samples, num_features = X.shape
        self.num_classes = len(np.unique(y))
        self.weights = np.zeros((num_features, self.num_classes))
        self.bias = np.zeros(self.num_classes)

        for _ in range(self.num_epochs):
            for batch_start in range(0, num_samples, self.batch_size):
                batch_end = min(batch_start + self.batch_size, num_samples)
                X_batch = X[batch_start:batch_end]
                y_batch = y[batch_start:batch_end]

                for idx, x_i in enumerate(X_batch):
                    for class_idx in range(self.num_classes):
                        y_val = 1 if y_batch[idx] == class_idx else -1
                        condition = y_val * (np.dot(x_i, self.weights[:, class_idx]) - self.bias[class_idx]) >= 1
                        if not condition:
                            self.weights[:, class_idx] -= self.learning_rate * (2 * self.lambda_param * self.weights[:, class_idx] - np.dot(x_i, y_val))
                            self.bias[class_idx] -= self.learning_rate * y_val

    def predict(self, X):
        approx = np.dot(X, self.weights) - self.bias
        return np.argmax(approx, axis=1)


In [97]:
# Train the SVM model
clf = SVM()
clf.fit(X_train, y_train)

In [98]:
predictions = clf.predict(X_test)

In [99]:
# Calculate accuracy
accuracy = accuracy_score(y_test, predictions)
print("Accuracy:", accuracy)

Accuracy: 0.6911111111111111


In [108]:
# Suppress warnings
warnings.filterwarnings("ignore", category=UserWarning)
# Print classification report
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.74      0.61      0.67       150
           1       0.65      0.67      0.66       150
           2       0.70      0.79      0.74       150

    accuracy                           0.69       450
   macro avg       0.69      0.69      0.69       450
weighted avg       0.69      0.69      0.69       450



In [110]:
# Preprocess the input text
test_text_processed = remove_duplicate_chars("การทำงานของร้านค้าช้ามาก กว่าจะส่งของช้ามากๆ")

# Transform the preprocessed text into TF-IDF numerical features
test_text_tfidf = vectorizer.transform([test_text_processed]).toarray()

# Predict the sentiment label using the trained SVM classifier
predicted_label = clf.predict(test_text_tfidf)

# Mapping of predicted label to sentiment
sentiment_mapping = {0: "Negative", 1: "Neutral", 2: "Positive"}

# Print the classification result
print("The text is classified as:", sentiment_mapping[predicted_label[0]])


The text is classified as: Negative
